<a href="https://colab.research.google.com/github/nilesh0109/CV2_SoSe_19/blob/master/Copy_of_cv2_ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import h5py

In [0]:
"""
unpickle function from CIFAR-10 website
"""
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='latin')
    return d

def create_dataset_from_files(files):
	rawdata = []
	labels = []
	for f in files:
		d = unpickle(f)
		rawdata.extend(d["data"])
		labels.extend(d["labels"])

	rawdata = np.array(rawdata)

	red = rawdata[:,:1024].reshape((-1,32,32))
	green = rawdata[:,1024:2048].reshape((-1,32,32))
	blue = rawdata[:,2048:].reshape((-1,32,32))

	data = np.stack((red,green,blue), axis=3)
	labels = np.array(labels)

	return data, labels

cifar_training_files = ['data_batch_{:d}'.format(i) for i in range(1,6)]
cifar_testing_files = ['test_batch']

train_data, train_labels = create_dataset_from_files(cifar_training_files)
test_data, test_labels = create_dataset_from_files(cifar_testing_files)

print(train_data.shape, train_data.dtype) # (50000, 32, 32, 3) -- 50000  images, each 32 x 32 with 3 color channels (UINT8)
print(train_labels.shape, train_labels.dtype) # 50000 labels, one for each image (INT64)

print(test_data.shape, test_data.dtype) # 10000 images, each 32 x 32 with 3 color channels (UINT8)
print(test_labels.shape, test_labels.dtype) # 10000 labels, one for each image (INT64)

(50000, 32, 32, 3) uint8
(50000,) int64
(10000, 32, 32, 3) uint8
(10000,) int64


In [0]:
hf = h5py.File('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
layer_groups = list(hf)
print(layer_groups)
#data = hf.get('/block1_conv1')
#vgg_weights
weights={}
for layer_grp in layer_groups:
  grp = hf.get(layer_grp)
  for k in grp.keys():
    print( k + ' shape: {}'.format(grp[k].shape))
    weights.update({k: grp[k].value})
print(weights.keys())

['layer_0', 'layer_1', 'layer_10', 'layer_11', 'layer_12', 'layer_13', 'layer_14', 'layer_15', 'layer_16', 'layer_17', 'layer_18', 'layer_19', 'layer_2', 'layer_20', 'layer_21', 'layer_22', 'layer_23', 'layer_24', 'layer_25', 'layer_26', 'layer_27', 'layer_28', 'layer_29', 'layer_3', 'layer_30', 'layer_31', 'layer_32', 'layer_33', 'layer_34', 'layer_35', 'layer_36', 'layer_4', 'layer_5', 'layer_6', 'layer_7', 'layer_8', 'layer_9']
param_0 shape: (64, 3, 3, 3)
param_1 shape: (64,)
param_0 shape: (256, 128, 3, 3)
param_1 shape: (256,)
param_0 shape: (256, 256, 3, 3)
param_1 shape: (256,)
param_0 shape: (256, 256, 3, 3)
param_1 shape: (256,)
param_0 shape: (512, 256, 3, 3)
param_1 shape: (512,)
param_0 shape: (512, 512, 3, 3)
param_1 shape: (512,)
param_0 shape: (512, 512, 3, 3)
param_1 shape: (512,)
param_0 shape: (512, 512, 3, 3)
param_1 shape: (512,)
param_0 shape: (512, 512, 3, 3)
param_1 shape: (512,)
param_0 shape: (512, 512, 3, 3)
param_1 shape: (512,)
param_0 shape: (64, 64, 3, 3)

In [0]:
#Training the Neural Network
def train_network(train_data, train_label, batch_size= 32, learning_rate=0.01, prediction_mode= False):
  imgs_placeholder = tf.placeholder(tf.uint8, shape=(None, 32, 32, 3))
  imgs_label_placeholder = tf.placeholder(tf.int64, shape=(None,))
  #preprocessed_imgs = tf.image.convert_image_dtype(imgs_placeholder, tf.float32)
  
  with tf.name_scope('preprocess') as scope:
    imgs = tf.image.convert_image_dtype(imgs_placeholder, tf.float32) * 255.0
    mean = tf.constant([123.68 , 116.779 , 103.939] ,dtype = tf.float32, shape =[1 , 1 , 1 , 3], name ='img_mean')
    imgs_normalized = imgs - mean
  
  with tf.name_scope('conv1_1') as scope:
    kernel = tf.Variable(initial_value = weights['layer0_conv1_W_1:0'], trainable = False, name ='weights')
    biases = tf.Variable(initial_value = weights['block1_conv1_b_1:0'], trainable = False, name ='biases')
    conv = tf.nn.conv2d(imgs_normalized, kernel, [1 , 1 , 1 , 1], padding ='SAME')
    out = tf.nn.bias_add(conv, biases)
    act11 = tf.nn.relu(out, name=scope)
    
  with tf.name_scope('conv1_2') as scope:
    kernel = tf.Variable(initial_value = weights['block1_conv2_W_1:0'], trainable = False, name ='weights')
    biases = tf.Variable(initial_value = weights['block1_conv2_b_1:0'], trainable = False, name ='biases')
    conv = tf.nn.conv2d(act11, kernel, [1 , 1 , 1 , 1], padding ='SAME')
    out = tf.nn.bias_add(conv, biases)
    act12 = tf.nn.relu(out, name=scope)
  
  with tf.name_scope('pool1') as scope:
    #pooling_kernel = tf.Variable(initial_value = weights['pool_1'], trainable = False, name ='weights')
    pool1 = tf.nn.max_pool(act12, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
  with tf.name_scope('conv2_1') as scope:
    kernel = tf.Variable(initial_value = weights['block2_conv1_W_1:0'], trainable = False, name ='weights')
    biases = tf.Variable(initial_value = weights['block2_conv1_b_1:0'], trainable = False, name ='biases')
    conv = tf.nn.conv2d(pool1, kernel, [1 , 1 , 1 , 1], padding ='SAME')
    out = tf.nn.bias_add(conv, biases)
    act21 = tf.nn.relu(out, name=scope)
    
  with tf.name_scope('conv2_2') as scope:
    kernel = tf.Variable(initial_value = weights['block2_conv2_W_1:0'], trainable = False, name ='weights')
    biases = tf.Variable(initial_value = weights['block2_conv2_b_1:0'], trainable = False, name ='biases')
    conv = tf.nn.conv2d(act21, kernel, [1 , 1 , 1 , 1], padding ='SAME')
    out = tf.nn.bias_add(conv, biases)
    act22 = tf.nn.relu(out, name=scope)

  with tf.name_scope('pool2') as scope:
    #pooling_kernel = tf.Variable(initial_value = weights['pool_2'], trainable = False, name ='weights')
    pool2 = tf.nn.max_pool(act22, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
  with tf.name_scope('conv3_1') as scope:
    kernel = tf.Variable(initial_value = weights['block3_conv1_W_1:0'], trainable = False, name ='weights')
    biases = tf.Variable(initial_value = weights['block3_conv1_b_1:0'], trainable = False, name ='biases')
    conv = tf.nn.conv2d(pool2, kernel, [1 , 1 , 1 , 1], padding ='SAME')
    out = tf.nn.bias_add(conv, biases)
    act31 = tf.nn.relu(out, name=scope)
    
  with tf.name_scope('conv3_2') as scope:
    kernel = tf.Variable(initial_value = weights['block3_conv2_W_1:0'], trainable = False, name ='weights')
    biases = tf.Variable(initial_value = weights['block3_conv2_b_1:0'], trainable = False, name ='biases')
    conv = tf.nn.conv2d(act31, kernel, [1 , 1 , 1 , 1], padding ='SAME')
    out = tf.nn.bias_add(conv, biases)
    act32 = tf.nn.relu(out, name=scope)
    
  with tf.name_scope('pool3') as scope:
    #pooling_kernel = tf.Variable(initial_value = weights['pool_3'], trainable = False, name ='weights')
    pool3 = tf.nn.max_pool(act32, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
  with tf.variable_scope('fc') as scope:
    flat = tf.layers.flatten(pool3)
    logits = tf.layers.dense(flat, units=10, reuse=tf.AUTO_REUSE)
    
   ######################### Define Loss function ################################
    loss =  tf.losses.sparse_softmax_cross_entropy(imgs_label_placeholder, logits)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
    minimizer_op = optimizer.minimize(loss)
    
    ################## Add ops to save and restore all the variables. ###############
    saver = tf.train.Saver()
    
    ######################### Accuracy check ################################
    predictions = tf.argmax(logits, axis=1)
    correct_pred = tf.equal(imgs_label_placeholder, predictions)
    accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))
    
    ################## Adding summary for consoling through tensorboard ###############
    loss_summary = tf.summary.scalar(name="loss", tensor=loss) #writes loss summary
    acc_summary = tf.summary.scalar(name="accuracy", tensor=accuracy) #writes accuracy summary
    
    
    ######################### Start Training ################################
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #saver.restore(sess, "model.ckpt")
        print('model restored')
        if(prediction_mode):
          print('prediction mode on')
          pred, a = sess.run([predictions, accuracy], {imgs_placeholder: train_data, imgs_label_placeholder: train_label})
          print('accuracy is ', a)
          print('pred is ', pred)
        else:
          print('Training mode on')
          for k in range(50):
              j = 0
              unseen_data = train_data[:]
              while(len(unseen_data)):
                  b_size = batch_size if batch_size < len(unseen_data) else len(unseen_data)
                  idx = np.random.choice( unseen_data.shape[0], b_size, replace=False )
                  indices_to_keep = [i for i in range(len(unseen_data)) if i not in idx]
                  unseen_data = unseen_data[indices_to_keep]
                  j = j + 1
                  training_data_batch = train_data[idx]
                  train_label_batch = train_label[idx]
                  _, l, acc, l_summary, a_summary =sess.run([minimizer_op, loss, accuracy, loss_summary, acc_summary], {imgs_placeholder: training_data_batch, imgs_label_placeholder: train_label_batch})
                  writer = tf.summary.FileWriter('./graphs', sess.graph)
                  writer.add_summary(l_summary, global_step= j) # writes loss summary
                  writer.add_summary(a_summary, global_step=j) # writes loss summary
                  #print('After iteration {} batch {} loss is {} and accuracy is {}%'.format(k, j, l, acc))

              print('*********************************************************************')
              print('After iteration {} loss is {} and accuracy is {}%'.format(k, l, acc*100))
              save_path = saver.save(sess, "model.ckpt".format(k))
    
  

In [0]:
train_network(train_data, train_labels, 64, 1e-06)

(?, 32, 32, 3)


KeyError: ignored